In [23]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from statsmodels.stats.multitest import multipletests
from scipy.stats import multivariate_t, multivariate_normal, norm, binom, kendalltau, pearsonr
import itertools
import json
from datetime import datetime
import os
import sqlite3
import pickle

# from google.colab import drive
# drive.mount('/content/drive')


# https://github.com/puolival/multipy


In [24]:
returns_path = '../data/russian_clear_returns.csv'
# returns_path = '/content/drive/My Drive/Диплом/russian_clear_returns.csv'

volume_path = '../data/russian_clear_volume.csv'
# volume_path = '/content/drive/My Drive/Диплом/russian_clear_volume.csv'


data = pd.read_csv(returns_path, sep=',')
data.set_index(data.columns[0], inplace=True)
data.rename_axis('Date', inplace=True)


In [25]:
data.head()


,ABRD,AFKS,AFLT,AKRN,ALRS,AMEZ,APTK,AQUA,ASSB,AVAN,...,VJGZ,VJGZP,VLHZ,VRSB,VSMO,VTBR,WTCM,WTCMP,YAKG,ZILL
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-05,-0.002497,0.019751,0.009392,-0.001337,0.008572,-0.008174,-0.013150,0.007576,0.033735,-0.009524,...,-0.001228,0.002497,-0.100796,0.003008,-0.011472,0.013618,-0.017506,-0.009615,0.018199,0.009425
2021-01-06,0.007472,-0.011073,0.003020,0.002005,0.013565,0.008853,0.000000,-0.005676,-0.010916,-0.004796,...,-0.004926,0.017306,-0.018385,0.002999,-0.005786,-0.005477,-0.002210,0.007220,-0.016364,0.020428
2021-01-08,-0.002484,0.005382,0.008462,0.000667,0.031498,0.002032,0.003775,0.013195,0.012121,0.000000,...,0.009828,0.002448,0.018385,0.000000,-0.002906,0.018271,0.006615,0.007168,-0.013844,0.007326
2021-01-11,-0.004988,0.026748,0.006772,0.001333,0.015999,-0.009517,-0.003048,-0.013195,-0.002413,0.004796,...,-0.001223,0.000000,0.010356,0.020741,-0.001942,0.024104,0.006572,-0.002384,0.012927,0.021661
2021-01-12,0.000000,0.030745,0.001349,-0.005679,0.015096,-0.006167,-0.000582,-0.005709,-0.008490,-0.009615,...,0.009744,0.009732,-0.010356,0.005848,-0.000972,0.007119,-0.006572,0.021252,0.007313,0.033366


In [26]:
data = data.loc[:'2021-12-31']

In [27]:
data

,ABRD,AFKS,AFLT,AKRN,ALRS,AMEZ,APTK,AQUA,ASSB,AVAN,...,VJGZ,VJGZP,VLHZ,VRSB,VSMO,VTBR,WTCM,WTCMP,YAKG,ZILL
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-05,-0.002497,0.019751,0.009392,-0.001337,0.008572,-0.008174,-0.013150,0.007576,0.033735,-0.009524,...,-0.001228,0.002497,-0.100796,0.003008,-0.011472,0.013618,-0.017506,-0.009615,0.018199,0.009425
2021-01-06,0.007472,-0.011073,0.003020,0.002005,0.013565,0.008853,0.000000,-0.005676,-0.010916,-0.004796,...,-0.004926,0.017306,-0.018385,0.002999,-0.005786,-0.005477,-0.002210,0.007220,-0.016364,0.020428
2021-01-08,-0.002484,0.005382,0.008462,0.000667,0.031498,0.002032,0.003775,0.013195,0.012121,0.000000,...,0.009828,0.002448,0.018385,0.000000,-0.002906,0.018271,0.006615,0.007168,-0.013844,0.007326
2021-01-11,-0.004988,0.026748,0.006772,0.001333,0.015999,-0.009517,-0.003048,-0.013195,-0.002413,0.004796,...,-0.001223,0.000000,0.010356,0.020741,-0.001942,0.024104,0.006572,-0.002384,0.012927,0.021661
2021-01-12,0.000000,0.030745,0.001349,-0.005679,0.015096,-0.006167,-0.000582,-0.005709,-0.008490,-0.009615,...,0.009744,0.009732,-0.010356,0.005848,-0.000972,0.007119,-0.006572,0.021252,0.007313,0.033366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24,0.000000,-0.000672,-0.004062,0.018632,0.013974,0.002174,-0.004488,0.037250,0.000580,-0.019418,...,-0.009744,-0.023605,0.006274,-0.126218,-0.014246,-0.002333,0.067823,0.021053,0.001329,0.003824
2021-12-27,-0.007702,0.010294,0.002033,0.014343,0.010783,-0.001630,0.000000,-0.030214,-0.010496,-0.014815,...,0.334232,0.335106,-0.014701,-0.019928,0.000000,0.013499,-0.014448,-0.004640,-0.006665,0.001271
2021-12-28,0.000000,0.045246,-0.002712,0.003948,0.029005,-0.017277,0.002396,0.025953,-0.014763,0.004963,...,-0.113703,-0.132556,0.015396,0.014532,0.000000,0.013836,-0.018888,-0.028304,-0.000892,0.065174


calculate **pearson** measure:
$$\frac{1}{2\sqrt{n-3}}(\ln{\frac{1+r_{i,j}}{1-r_{i,j}}} - \ln{\frac{1+\gamma_0}{1-\gamma_0}})$$
where:
$$r_{i,j} = \frac{\sum_{t=1}^{n}{({x_i(t) - \bar{x_i}})({x_j(t) - \bar{x_j}})}}{\sqrt{\sum_{t=1}^{n}{({x_i(t) - \bar{x_i}})^2}}\sqrt{\sum_{t=1}^{n}{({x_j(t) - \bar{x_j}})^2}}}$$

In [112]:
def calculate_pearson_measure(ret_i, ret_j, gamma_0):
    # val_i = np.array(ret_i) - np.mean(ret_i)
    # val_j = np.array(ret_j) - np.mean(ret_j)
    # numenator_r = np.sum(val_i * val_j)
    # denumerator_r = (np.sum(val_i**2)*np.sum(val_j**2))**0.5
    # r_ij = numenator_r / denumerator_r
    r_ij = pearsonr(ret_i, ret_j)[0]

   #  value = (1 / (2 * (len(ret_i) - 3)**0.5)) * (np.log((1+r_ij) /
   #                                                     (1 - r_ij)) - np.log((1 + gamma_0) / (1 - gamma_0)))
    # print((np.log((1+r_ij) /
    #                     (1 - r_ij)) - np.log((1 + gamma_0) / (1 - gamma_0))))
    value = 1 / (2 * np.sqrt(len(ret_i) - 3)) * (np.log((1+r_ij) /
                                             (1 - r_ij)) - np.log((1 + gamma_0) / (1 - gamma_0))) / 2
    return value
    # return pearsonr(ret_i, ret_j)[0]


calculate **Kendall** measure:
$$\sqrt{\frac{9n(n-1)}{2(2n+5)}}(T_{ij}-\gamma_0)$$
where:
$$T_{i,j} = \frac{1}{n(n-1)}\sum_{s=1}^{n}\sum_{t=1}^{n}sign((x_i(t)-x_i(s))(x_j(t)-x_j(s)))$$

In [29]:
def calculate_kendall_measure(ret_i, ret_j, gamma_0):
    n = len(ret_i)
    # signs = []
    # for i in range(n):
    #     for j in range(i+1, n):
    #         signs.append((ret_i[i] - ret_i[j]) * (ret_j[i] - ret_j[j]) > 0)
    # T_ij = 2 / (n * (n - 1)) * np.sum(signs)
    T_ij = kendalltau(ret_i, ret_j)[0]
    value = np.sqrt((9 * n * (n - 1)) / (2*(2*n+5)) ) * (T_ij - gamma_0)
    return value

In [30]:
def calculate_sign_measure(ret_i, ret_j, gamma_0=None):
    ret_i_mean = np.mean(ret_i)
    ret_j_mean = np.mean(ret_j)
    return np.sum([(ret_i[i] - ret_i_mean) * ( ret_j[i] - ret_j_mean) >= 0 for i in range(len(ret_i))])

# Holm

In [31]:
def get_hypotheses(data_np, measure, gamma_0):
    data_np = np.array(data_np)
    combinations = list(itertools.combinations(range(len(data_np.T)), 2))
    hypotheses = [(measure(data_np[:, x], data_np[:, y], gamma_0), (x, y)) for (x, y) in combinations]

    return hypotheses

In [32]:
def get_normal_constant(alpha):
    return norm.cdf(alpha)

def get_binom_constant(n, p0, alpha):
    lo, hi = 0, n
    while lo < hi:
        mid = (lo + hi) // 2
        if binom.cdf(mid, n, p0) >= alpha:
            hi = mid
        else:
            lo = mid + 1
    return lo


In [1364]:
def holm_procedure(data, gamma_0, alpha, hypotheses = None, measure=calculate_pearson_measure):


    if hypotheses == None:
        data_np = np.array(data)
        hypotheses = get_hypotheses(data_np, measure, gamma_0)

    hypotheses.sort()
    index = None
    n = data.shape[0]
    m = len(hypotheses)
    for i, (value, (k, j)) in enumerate(hypotheses):
        # вычисляем пороговый уровень значимости для текущей гипотезы
        # adjusted_alpha = 1 - alpha / (m - i + 1)
        adjusted_alpha = alpha / (m - i + 1)

        if measure == calculate_sign_measure:
            threshold = get_binom_constant(n, gamma_0, adjusted_alpha)
        else:
            threshold = get_normal_constant(adjusted_alpha) / n

        if value >= threshold:
            index = i
            break

    if index == None:
        strongly_connected = []
    else:
        strongly_connected = [i for (_, i) in hypotheses[index:]]

    return strongly_connected

In [844]:
data_short = data[data.columns[:50]]

# Hochberg

In [847]:
def hochberg_procedure(data, gamma_0, alpha,hypotheses = None, measure=calculate_pearson_measure):

    if hypotheses == None:
        data_np = np.array(data)
        hypotheses = get_hypotheses(data_np, measure, gamma_0)

    hypotheses.sort(reverse=True)

    m = len(hypotheses)
    n = data.shape[0]
    strongly_connected = []
    index = None

    for i, (value, (k, j)) in enumerate(hypotheses):
        adjusted_alpha = 1 - alpha / (i + 1)

        if measure == calculate_sign_measure:
            threshold = get_binom_constant(n, gamma_0, adjusted_alpha)
        else:
            threshold = get_normal_constant(adjusted_alpha) / n

        if value < threshold:
            index = i
            break

    if index == None:
        strongly_connected = [i for (_, i) in hypotheses[:]]
    else:
        strongly_connected = [i for (_, i) in hypotheses[:index]]

    return strongly_connected

# Benjamini-Hochberg

In [849]:
def benjamini_hochberg_procedure(data, gamma_0, alpha,hypotheses=None, measure = pearsonr):

    if hypotheses == None:
        data_np = np.array(data)
        hypotheses = get_hypotheses(data_np, measure, gamma_0)
    hypotheses.sort()

    m = len(hypotheses)
    n = data.shape[0]
    index = None


    for i, (value, (i, j)) in enumerate(hypotheses):
        adjusted_alpha = 1 - (i+1) * alpha / m
        
        if measure == calculate_sign_measure:
            threshold = get_binom_constant(n, gamma_0, adjusted_alpha)
        else:
            threshold = get_normal_constant(adjusted_alpha)  / n
        
        if value >= threshold:
            index = i
            break

    if index == None:
        strongly_connected = []
    else:
        strongly_connected = [i for (_, i) in hypotheses[index:]]

    return strongly_connected

# Data generation

In [854]:
def generate_data(sample_data, gamma, size, df = 3):
    mean = sample_data.mean()
    cov = np.cov(sample_data.T)
    # normal = np.random.multivariate_normal(mean, cov, size=size)
    # normal = np.random.multivariate_normal(mean, cov, size=size)
    normal = multivariate_normal.rvs(mean, cov, size)
    t_dist = multivariate_t.rvs(df=df, loc=mean, shape=cov, size=size)

    return gamma * normal + (1-gamma) * t_dist

# Tests

In [857]:
data.columns[121]

'SBERP'

In [1389]:
volume = pd.read_csv(volume_path, sep=',', index_col='TRADEDATE').mean()
volume = volume.sort_values()[::-1]

In [1390]:
volume[:50].index

Index(['TATN', 'TATNP', 'TGKB', 'TGKN', 'ENRU', 'UPRO', 'MRKY', 'SNGS', 'MAGN',
       'SNGSP', 'RGSS', 'UNAC', 'TTLK', 'AFLT', 'MSNG', 'NLMK', 'DVEC', 'MOEX',
       'SBERP', 'ROSN', 'MSRS', 'NMTP', 'MTLR', 'LIFE', 'DSKY', 'MTSS', 'LSNG',
       'RTKM', 'STSBP', 'FESH', 'MRSB', 'NVTK', 'AMEZ', 'CHMF', 'MTLRP',
       'FLOT', 'RASP', 'TRMK', 'SIBN', 'LKOH', 'RTSB', 'ISKJ', 'KZOSP', 'ASSB',
       'SAGO', 'BLNG', 'BSPB', 'CNTLP', 'SELG', 'PIKK'],
      dtype='object')

In [1448]:
Gammas = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
# Gammas = [ 0.6, 0.7, 0.8, 0.9, 1]
# Gammas = [0, 0.5, 1]

Sample_size = [100, 500, 1000]
# Sample_size = [500]

Stocks_number = [5, 10, 20]
# Gamma_0 = [0.1, 0.3, 0.5, 0.7]
Gamma_0 = [0.1, 0.3, 0.6, 0.75]
# Num_of_retries = 100
Num_of_retries = 50
Measure = [calculate_pearson_measure, calculate_kendall_measure, calculate_sign_measure]
# Measure = [calculate_pearson_measure, calculate_sign_measure]
Measure = [calculate_sign_measure]
alpha = 0.1

In [1421]:
len(Gammas)*len(Sample_size)*len(Stocks_number)*len(Gamma_0)*Num_of_retries*len(Measure) * 3

44550

In [1449]:
def tests_runner(data,
                volume,
                Gammas,
                Sample_size,
                Stocks_number,
                Gamma_0,
                Num_of_retries, 
                Measure,
                alpha):

    base_path = "../tests"
    tests_file = datetime.now().strftime(f"{base_path}/test__%d_%m__%H_%M_%S.json")
    with open(tests_file, "w") as outfile:
        outfile.write('[')
        
    true_values_file = datetime.now().strftime(f"{base_path}/true_values__%d_%m__%H_%M_%S.json")
    with open(true_values_file, "w") as outfile:
        outfile.write('[')
    
    for gamma in tqdm(Gammas):
        for sample_size in Sample_size:
            for stocks_number in Stocks_number:
                for gamma_0 in Gamma_0:
                    for measure in Measure:
                        # True values
                        cols = volume.index[:stocks_number]
                        X = data[cols]
                        true_hypotheses = get_hypotheses(X, measure, gamma_0)
                        holm = holm_procedure(X, gamma_0, alpha, true_hypotheses, measure)
                        hochberg = hochberg_procedure(X, gamma_0, alpha, true_hypotheses, measure)
                        benjamini_hochberg = benjamini_hochberg_procedure(X, gamma_0, alpha, true_hypotheses, measure)
                        holm = {
                            'procedure': 'holm',
                            'measure': measure.__name__,
                            'gamma_0': gamma_0,
                            'stocks_number': stocks_number,
                            'sample_size': sample_size,
                            'gamma': gamma,
                            'nodes': holm
                            }
                        
                        hochberg = {
                            'procedure': 'hochberg',
                            'measure': measure.__name__,
                            'gamma_0': gamma_0,
                            'stocks_number': stocks_number,
                            'sample_size': sample_size,
                            'gamma': gamma,
                            'nodes': hochberg
                            }

                        benjamini_hochberg = {
                            'procedure': 'benjamini_hochberg',
                            'measure': measure.__name__,
                            'gamma_0': gamma_0,
                            'stocks_number': stocks_number,
                            'sample_size': sample_size,
                            'gamma': gamma,
                            'nodes': benjamini_hochberg
                            }

                        holm_json = json.dumps(holm)
                        hochberg_json = json.dumps(hochberg)
                        benjamini_hochberg_json = json.dumps(benjamini_hochberg)

                        with open(true_values_file, "a") as outfile:
                            outfile.write(holm_json + ',')
                            outfile.write(hochberg_json + ',')
                            outfile.write(benjamini_hochberg_json + ',')

                        # Generated values
                        for i in range(Num_of_retries):
                            cols = volume[:stocks_number].index
                            values = data[cols]
                            X = generate_data(values, gamma, sample_size, 3)
                            hypotheses = get_hypotheses(X, measure, gamma_0)

                            holm = holm_procedure(X, gamma_0, alpha, hypotheses, measure)
                            hochberg = hochberg_procedure(X, gamma_0, alpha, hypotheses, measure)
                            benjamini_hochberg = benjamini_hochberg_procedure(X, gamma_0, alpha, hypotheses, measure)
                            holm = {
                                'procedure': 'holm',
                                'measure': measure.__name__,
                                'gamma_0': gamma_0,
                                'stocks_number': stocks_number,
                                'sample_size': sample_size,
                                'gamma': gamma,
                                'retry': i,
                                'nodes': holm
                                }
                            

                            hochberg = {
                                'procedure': 'hochberg',
                                'measure': measure.__name__,
                                'gamma_0': gamma_0,
                                'stocks_number': stocks_number,
                                'sample_size': sample_size,
                                'gamma': gamma,
                                'retry': i,
                                'nodes': hochberg
                                }

                            benjamini_hochberg = {
                                'procedure': 'benjamini_hochberg',
                                'measure': measure.__name__,
                                'gamma_0': gamma_0,
                                'stocks_number': stocks_number,
                                'sample_size': sample_size,
                                'gamma': gamma,
                                'retry': i,
                                'nodes': benjamini_hochberg
                                }

                            holm_json = json.dumps(holm)
                            hochberg_json = json.dumps(hochberg)
                            benjamini_hochberg_json = json.dumps(benjamini_hochberg)

                            # Writing to sample.json
                            with open(tests_file, "a") as outfile:
                                outfile.write(holm_json + ',')
                                outfile.write(hochberg_json + ',')
                                outfile.write(benjamini_hochberg_json + ',')
        
    with open(tests_file, 'rb+') as filehandle:
        filehandle.seek(-1, os.SEEK_END)
        filehandle.truncate()

    with open(tests_file, 'a') as filehandle:
        filehandle.write(']')

    with open(true_values_file, 'rb+') as filehandle:
        filehandle.seek(-1, os.SEEK_END)
        filehandle.truncate()

    with open(true_values_file, 'a') as filehandle:
        filehandle.write(']')
    
    return (tests_file, true_values_file)

In [1450]:
def new_true_values_runner(data,
                           newData,
                           volume,
                           Stocks_number,
                           Gamma_0,
                           Measure,
                           alpha):

    base_path = "../tests"
        
    new_true_values_file = datetime.now().strftime(f"{base_path}/new_true_values__%d_%m__%H_%M_%S.json")
    with open(new_true_values_file, "w") as outfile:
        outfile.write('[')
    
    true_values_file = datetime.now().strftime(f"{base_path}/true_values__%d_%m__%H_%M_%S.json")
    with open(true_values_file, "w") as outfile:
        outfile.write('[')

    for stocks_number in Stocks_number:
        for gamma_0 in Gamma_0:
            for measure in Measure:
                # True values
                cols = volume[:stocks_number].index
                X = data[cols]
                hochberg = hochberg_procedure(X, gamma_0, alpha, measure=measure)

                X = newData[cols]
                new_hochberg = hochberg_procedure(X, gamma_0, alpha, measure=measure)
                
                hochberg_stat = {
                    'procedure': 'hochberg',
                    'measure': measure.__name__,
                    'gamma_0': gamma_0,
                    'stocks_number': stocks_number,
                    'nodes': hochberg
                    }
                
                new_hochberg_stat = {
                    'procedure': 'hochberg',
                    'measure': measure.__name__,
                    'gamma_0': gamma_0,
                    'stocks_number': stocks_number,
                    'nodes': new_hochberg
                    }

                hochberg_json = json.dumps(hochberg_stat)
                new_hochberg_json = json.dumps(new_hochberg_stat)
                with open(true_values_file, "a") as outfile:
                    outfile.write(hochberg_json + ',')

                with open(new_true_values_file, "a") as outfile:
                    outfile.write(new_hochberg_json + ',')


    with open(new_true_values_file, 'rb+') as filehandle:
        filehandle.seek(-1, os.SEEK_END)
        filehandle.truncate()

    with open(new_true_values_file, 'a') as filehandle:
        filehandle.write(']')

    with open(true_values_file, 'rb+') as filehandle:
        filehandle.seek(-1, os.SEEK_END)
        filehandle.truncate()

    with open(true_values_file, 'a') as filehandle:
        filehandle.write(']')
    
    return (true_values_file, new_true_values_file)

In [1451]:
(testsFile, trueValuesFile) = tests_runner(data,
                                           volume,
                                           Gammas,
                                           Sample_size,
                                           Stocks_number,
                                           Gamma_0,
                                           Num_of_retries,
                                           Measure,
                                           alpha)
print(f"tests: {testsFile}")
print(f"true values: {trueValuesFile}")

100%|██████████| 11/11 [29:20<00:00, 160.08s/it]

tests: ../tests/test__31_05__17_30_02.json
true values: ../tests/true_values__31_05__17_30_02.json


# Metrics 

In [1452]:
tests_file = testsFile
# tests_file = '../tests/result_test.json'
# tests_file = '../tests/test__25_05__21_41_35.json' # русские акции 1 час, итоговые метрики

# tests_file = '../tests/test__04_05__13_45_11.json' # русские акции 10 часов
# tests_file = '..\\tests\\test__29_04__21_55_33.json' # американские акции

trueValues_file = trueValuesFile

# tests_file = '../tests/test__31_05__12_10_59.json'
# trueValues_file = '../tests/true_values__31_05__12_10_59.json'


# trueValues_file = '../tests/result_true.json'
# trueValues_file = '../tests/true_values__25_05__21_41_35.json' # русские акции 1 час, итоговые метрики

# trueValues_file = '../tests/true_values__04_05__13_45_11.json' # русские акции 10 часов
# tests_file = '..\\tests\\true_values__30_04__15_30_02.json' # американские акции

In [1453]:
with open(tests_file, "r") as inputFile:
    tests = json.load(inputFile)

with open(trueValues_file, "r") as inputFile:
    true_values = json.load(inputFile)

In [1454]:
print(len(tests), len(true_values))

59400 1188


* включение всех истинных ребер в множество на сгенерированный данных
* среднее число ребер
* среднее число ребер, которые есть в истинном множестве, но нет в сгенерированном

In [1455]:
def calc_statistics(db_name, tests_table_name, true_values):
    '''
    approximate time for calulation: 22h
    single iteration: about 20s
    '''
    
    for true_val in true_values:
        conn = sqlite3.connect(db_name)

        cur = conn.cursor()

        cur.execute(f'SELECT * FROM {tests_table_name} where procedure = ? and measure = ? and gamma_0 = ? and stocks_number = ? and sample_size = ? and gamma = ?',
                    (true_val['procedure'], true_val['measure'], true_val['gamma_0'], true_val['stocks_number'], true_val['sample_size'], true_val['gamma']))

        rows = cur.fetchall()


In [1456]:
objects = {}
for test in tests:
    objects.setdefault(test['procedure'], {}).setdefault(test['measure'], {}).setdefault(test['gamma_0'], {}).setdefault(test['stocks_number'], {}).setdefault(test['sample_size'], {}).setdefault(test['gamma'], []).append(test['nodes'])


In [1457]:
true_objects = {}
for true_val in true_values:
    true_objects.setdefault(true_val['procedure'], {}).setdefault(true_val['measure'], {}).setdefault(true_val['gamma_0'], {}).setdefault(true_val['stocks_number'], {}).setdefault(true_val['sample_size'], {}).setdefault(true_val['gamma'], []).append(true_val['nodes'])

In [1459]:
def get_statistics(statistics, procedure, measure, stocks_number, sample_size):
    index = [i for i in statistics[procedure][measure].keys()]
    table = pd.DataFrame(index=index, columns=statistics[procedure][measure][index[0]][stocks_number][sample_size].keys())
    table.rename_axis(index='gamma_0', columns='gamma', inplace=True)

    for i in statistics[procedure][measure].keys():
        for j in statistics[procedure][measure][i][stocks_number][sample_size].keys():
            table[j][i] = statistics[procedure][measure][i][stocks_number][sample_size][j]

    return table

In [1460]:
statistics = {}

for true_val in true_values:
    accepted_list = []

    for test in objects[true_val['procedure']][true_val['measure']][true_val['gamma_0']][true_val['stocks_number']][true_val['sample_size']][true_val['gamma']]:
        if len(true_val['nodes']) != 0:
            # print(true_val, test, set([(i[0], i[1]) for i in test]) & set([(i[0], i[1]) for i in true_val['nodes']]))
            # if true_val['measure'] == 'calculate_sign_measure' and true_val['gamma_0'] == 0.6:
            #     print(true_val, test, len(set([(i[0], i[1]) for i in test]) & set([(i[0], i[1]) for i in true_val['nodes']])) / len(true_val['nodes']))


            # accepted_list.append(len(set([(i[0], i[1]) for i in test]) & set([(i[0], i[1]) for i in true_val['nodes']])) / len(true_val['nodes']))
            accepted_list.append(len(set([(i[0], i[1]) for i in test]) & set([(i[0], i[1]) for i in true_val['nodes']])) == len(true_val['nodes']))

    statistics.setdefault(true_val['procedure'], {}).setdefault(true_val['measure'], {}).setdefault(true_val['gamma_0'], {}).setdefault(true_val['stocks_number'], {}).setdefault(true_val['sample_size'], {})[true_val['gamma']] = np.mean(accepted_list)


In [1461]:
edges_statistics = {}

for true_val in true_values:
    accepted_list = []

    for test in objects[true_val['procedure']][true_val['measure']][true_val['gamma_0']][true_val['stocks_number']][true_val['sample_size']][true_val['gamma']]:
        accepted_list.append(len(test))

    edges_statistics.setdefault(true_val['procedure'], {}).setdefault(true_val['measure'], {}).setdefault(true_val['gamma_0'], {}).setdefault(true_val['stocks_number'], {}).setdefault(true_val['sample_size'], {})[true_val['gamma']] = np.mean(accepted_list)

In [1462]:
not_included_edges = {}

for true_val in true_values:
    accepted_list = []
    
    for test in objects[true_val['procedure']][true_val['measure']][true_val['gamma_0']][true_val['stocks_number']][true_val['sample_size']][true_val['gamma']]:
        diff = set([(i[0], i[1]) for i in true_val['nodes']]).difference(set([(j[0], j[1]) for j in test]))
        accepted_list.append(len(diff))

    not_included_edges.setdefault(true_val['procedure'], {}).setdefault(true_val['measure'], {}).setdefault(true_val['gamma_0'], {}).setdefault(true_val['stocks_number'], {}).setdefault(true_val['sample_size'], {})[true_val['gamma']] = np.mean(accepted_list)

#### Some statistics

In [ ]:
calculate_kendall_measure

In [ ]:
pearson_edge_table = get_statistics(edges_statistics, 'holm', 'calculate_sign_measure', 5, 1000) 
print(pearson_edge_table[pearson_edge_table.columns[[0, 2, 4, 6, 8, 10]]].to_latex(
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                  ))
pearson_edge_table

In [ ]:
calculate_sign_measure

In [1499]:
for j in ['holm', 'hochberg', 'benjamini_hochberg']:
    print(j)
    for i in [0.1, 0.3, 0.6, 0.75]:
        print(len(true_objects[j]['calculate_sign_measure'][i][20][1000][0][0]))

holm
190
190
183
7
hochberg
190
190
9
1
benjamini_hochberg
188
188
172
190


In [1503]:
pearson_table = get_statistics(statistics, 'holm', 'calculate_sign_measure', 10, 1000)
print(pearson_table.to_latex(
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                  float_format="{:.2f}".format
                  ))
pearson_table 

\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
\textbf{gamma} &  0.0 &  0.1 &  0.2 &  0.3 &  0.4 &  0.5 &  0.6 &  0.7 &  0.8 &  0.9 &  1.0 \\
\textbf{gamma\_0} &      &      &      &      &      &      &      &      &      &      &      \\
\midrule
\textbf{0.10   } & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 \\
\textbf{0.30   } & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 \\
\textbf{0.60   } & 0.38 & 0.48 & 0.38 & 0.36 & 0.36 & 0.50 & 0.54 & 0.48 & 0.34 & 0.46 & 0.40 \\
\textbf{0.75   } & 0.06 & 0.14 & 0.32 & 0.10 & 0.10 & 0.12 & 0.16 & 0.08 & 0.10 & 0.18 & 0.10 \\
\bottomrule
\end{tabular}



gamma,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gamma_0,,,,,,,,,,,
0.10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0.30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0.60,0.38,0.48,0.38,0.36,0.36,0.5,0.54,0.48,0.34,0.46,0.4
0.75,0.06,0.14,0.32,0.1,0.1,0.12,0.16,0.08,0.1,0.18,0.1


In [1504]:
print(', '.join(volume.index[:20]))

TATN, TATNP, TGKB, TGKN, ENRU, UPRO, MRKY, SNGS, MAGN, SNGSP, RGSS, UNAC, TTLK, AFLT, MSNG, NLMK, DVEC, MOEX, SBERP, ROSN


In [1486]:
len(true_objects['holm']['calculate_sign_measure'][0.75][20][1000][0][0])

7

In [1467]:
holm_procedure(data, 0.5, 0.1, measure=calculate_kendall_measure)

[(72, 97), (62, 112), (87, 88), (19, 97), (132, 133), (120, 121)]

In [1468]:
sts = {}
for i in objects['holm']['calculate_sign_measure'][0.6][20][500][0]:
    for j in set([(i[0],i[1]) for i in true_objects['holm']['calculate_sign_measure'][0.6][20][500][0][0]]).difference([(k[0], k[1]) for k in i]):
        if j[0] in sts.keys():
            sts[j[0]] += 1
        else:
            sts[j[0]] = 1
        
        if j[1] in sts.keys():
            sts[j[1]] += 1
        else:
            sts[j[1]] = 1

In [1469]:
ssts = {k: v for k, v in sorted(sts.items(), key=lambda item: item[1])}
print([volume.index[i] for i in ssts.keys()])
ssts

['ROSN', 'SBERP', 'ENRU', 'MAGN', 'TGKB', 'TTLK', 'NLMK', 'MSNG', 'TGKN', 'MOEX', 'TATNP', 'SNGS', 'UPRO', 'MRKY', 'SNGSP', 'TATN', 'RGSS', 'UNAC', 'DVEC']


{19: 2,
 18: 2,
 4: 3,
 8: 4,
 2: 4,
 12: 5,
 15: 5,
 14: 7,
 3: 7,
 17: 7,
 1: 8,
 7: 10,
 5: 13,
 6: 13,
 9: 14,
 0: 14,
 10: 21,
 11: 25,
 16: 42}

In [ ]:
pearson_not_included_edge_table = get_statistics(not_included_edges, 'holm', 'calculate_pearson_measure', 50, 1000)
print(pearson_not_included_edge_table[pearson_not_included_edge_table.columns[[0, 2, 4, 6, 8, 10]]].to_latex(
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                  ))
pearson_not_included_edge_table

\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
\textbf{gamma} &    0.0 &    0.2 &    0.4 &    0.6 &   0.8 &   1.0 \\
\textbf{gamma\_0} &        &        &        &        &       &       \\
\midrule
\textbf{0.10   } &  47.57 &  50.33 &  43.07 &  35.25 &  5.69 &  2.18 \\
\textbf{0.50   } &    0.9 &    0.8 &   0.88 &   0.42 &  0.04 &  0.04 \\
\textbf{0.75   } &    0.0 &    0.0 &    0.0 &    0.0 &   0.0 &   0.0 \\
\bottomrule
\end{tabular}



gamma,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gamma_0,,,,,,,,,,,
0.10,47.57,50.66,50.33,50.18,43.07,33.44,35.25,13.18,5.69,2.03,2.18
0.50,0.9,0.95,0.8,0.64,0.88,0.86,0.42,0.19,0.04,0.04,0.04
0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
kendall_not_included_edge_table = get_statistics(not_included_edges, 'holm', 'calculate_kendall_measure', 50, 1000) 
print(kendall_not_included_edge_table[kendall_not_included_edge_table.columns[[0, 2, 4, 6, 8, 10]]].to_latex(
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                  ))
kendall_not_included_edge_table

\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
\textbf{gamma} &     0.0 &     0.2 &     0.4 &    0.6 &     0.8 &     1.0 \\
\textbf{gamma\_0} &         &         &         &        &         &         \\
\midrule
\textbf{0.10   } &  188.24 &  188.55 &  187.28 &  191.9 &  192.92 &  190.03 \\
\textbf{0.50   } &    0.99 &    0.98 &    0.97 &   0.99 &     1.0 &     1.0 \\
\textbf{0.75   } &     0.0 &     0.0 &     0.0 &    0.0 &     0.0 &     0.0 \\
\bottomrule
\end{tabular}



gamma,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gamma_0,,,,,,,,,,,
0.10,188.24,188.51,188.55,189.26,187.28,189.43,191.9,192.47,192.92,189.93,190.03
0.50,0.99,0.97,0.98,0.98,0.97,0.99,0.99,0.98,1.0,0.99,1.0
0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
sign_not_included_edge_table = get_statistics(not_included_edges, 'benjamini_hochberg', 'calculate_sign_measure', 20, 500)
print(sign_not_included_edge_table[sign_not_included_edge_table.columns[[0, 2, 4, 6, 8, 10]]].to_latex(
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                  ))
sign_not_included_edge_table

\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
\textbf{gamma} &   0.0 &   0.2 &   0.4 &   0.6 &   0.8 &   1.0 \\
\textbf{gamma\_0} &       &       &       &       &       &       \\
\midrule
\textbf{0.10   } &  6.79 &  7.24 &  6.92 &   6.6 &  7.35 &  6.98 \\
\textbf{0.50   } &  2.48 &  2.17 &  2.31 &  2.27 &  2.26 &  2.67 \\
\textbf{0.75   } &  0.55 &  0.34 &   0.3 &  0.37 &  0.44 &  0.52 \\
\bottomrule
\end{tabular}



gamma,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gamma_0,,,,,,,,,,,
0.10,6.79,6.68,7.24,6.63,6.92,6.77,6.6,6.8,7.35,7.06,6.98
0.50,2.48,2.7,2.17,2.31,2.31,2.05,2.27,2.12,2.26,2.37,2.67
0.75,0.55,0.31,0.34,0.38,0.3,0.49,0.37,0.39,0.44,0.38,0.52


In [ ]:
edges_statistics.keys()

dict_keys(['holm', 'hochberg', 'benjamini_hochberg'])

In [ ]:
pearson_edge_table = get_statistics(edges_statistics, 'benjamini_hochberg', 'calculate_sign_measure', 20, 500) 
print(pearson_edge_table[pearson_edge_table.columns[[0, 2, 4, 6, 8, 10]]].to_latex(
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                  ))
pearson_edge_table

\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
\textbf{gamma} &     0.0 &     0.2 &     0.4 &     0.6 &     0.8 &     1.0 \\
\textbf{gamma\_0} &         &         &         &         &         &         \\
\midrule
\textbf{0.10   } &  177.16 &  176.76 &  177.08 &  177.45 &  176.63 &  176.79 \\
\textbf{0.50   } &  186.09 &  186.35 &  186.16 &  186.44 &  186.36 &  185.86 \\
\textbf{0.75   } &  189.28 &  189.58 &  189.58 &  189.54 &  189.44 &  189.34 \\
\bottomrule
\end{tabular}



gamma,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gamma_0,,,,,,,,,,,
0.10,177.16,177.54,176.76,177.62,177.08,177.57,177.45,177.32,176.63,177.25,176.79
0.50,186.09,185.76,186.35,186.2,186.16,186.59,186.44,186.4,186.36,186.27,185.86
0.75,189.28,189.56,189.58,189.52,189.58,189.44,189.54,189.42,189.44,189.5,189.34


In [ ]:
kendall_edge_table = get_statistics(edges_statistics, 'holm', 'calculate_kendall_measure', 50, 1000)
print(kendall_edge_table[kendall_edge_table.columns[[0, 2, 4, 6, 8, 10]]].to_latex(
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                  ))
kendall_edge_table

\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
\textbf{gamma} &     0.0 &    0.2 &     0.4 &     0.6 &     0.8 &     1.0 \\
\textbf{gamma\_0} &         &        &         &         &         &         \\
\midrule
\textbf{0.10   } &  684.91 &  684.6 &  684.84 &  678.48 &  675.46 &  678.64 \\
\textbf{0.50   } &    2.13 &   2.15 &    2.13 &    2.03 &    2.06 &    2.04 \\
\textbf{0.75   } &     2.0 &    2.0 &     2.0 &     2.0 &     2.0 &     2.0 \\
\bottomrule
\end{tabular}



gamma,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gamma_0,,,,,,,,,,,
0.10,684.91,683.97,684.6,682.69,684.84,681.92,678.48,676.68,675.46,679.55,678.64
0.50,2.13,2.14,2.15,2.08,2.13,2.12,2.03,2.09,2.06,2.03,2.04
0.75,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [ ]:
sign_edge_table = get_statistics(edges_statistics, 'holm', 'calculate_sign_measure', 50, 1000)
print(sign_edge_table[sign_edge_table.columns[[0, 2, 4, 6, 8, 10]]].to_latex(
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                  ))
sign_edge_table

\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
\textbf{gamma} &     0.0 &     0.2 &     0.4 &     0.6 &     0.8 &     1.0 \\
\textbf{gamma\_0} &         &         &         &         &         &         \\
\midrule
\textbf{0.10   } &  327.63 &  321.13 &  322.71 &  326.08 &  322.42 &  319.66 \\
\textbf{0.50   } &   12.58 &   12.02 &   12.29 &   12.37 &   12.33 &   12.32 \\
\textbf{0.75   } &    2.02 &    2.06 &    2.02 &    2.05 &    2.04 &    2.07 \\
\bottomrule
\end{tabular}



gamma,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gamma_0,,,,,,,,,,,
0.10,327.63,321.65,321.13,322.21,322.71,323.41,326.08,320.87,322.42,329.08,319.66
0.50,12.58,12.13,12.02,12.71,12.29,12.47,12.37,12.09,12.33,12.22,12.32
0.75,2.02,2.06,2.06,2.0,2.02,2.03,2.05,2.01,2.04,2.0,2.07


In [ ]:
statistics.keys()

dict_keys(['holm', 'hochberg', 'benjamini_hochberg'])

In [ ]:
pearson_table = get_statistics(statistics, 'holm', 'calculate_sign_measure', 20, 500)
print(pearson_table.to_latex(
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                  ))
pearson_table

\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
\textbf{gamma} &  0.0 &  0.1 &  0.2 &  0.3 &  0.4 &  0.5 &  0.6 &  0.7 &  0.8 &  0.9 &   1.0 \\
\textbf{gamma\_0} &      &      &      &      &      &      &      &      &      &      &       \\
\midrule
\textbf{0.10   } &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &   0.0 \\
\textbf{0.50   } &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.01 \\
\textbf{0.75   } &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &   1.0 \\
\bottomrule
\end{tabular}



gamma,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gamma_0,,,,,,,,,,,
0.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01
0.75,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
pearson_table = get_statistics(statistics, 'hochberg', 'calculate_pearson_measure', 20, 1000)
print(pearson_table.to_latex(
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                  ))
pearson_table

\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
\textbf{gamma} &   0.0 &   0.1 &   0.2 &   0.3 &   0.4 &   0.5 &   0.6 &   0.7 &   0.8 &   0.9 &   1.0 \\
\textbf{gamma\_0} &       &       &       &       &       &       &       &       &       &       &       \\
\midrule
\textbf{0.10   } &  0.02 &  0.04 &  0.04 &  0.05 &  0.05 &  0.08 &  0.18 &  0.35 &  0.52 &  0.61 &  0.74 \\
\textbf{0.50   } &  0.64 &  0.65 &   0.6 &  0.69 &  0.78 &  0.69 &  0.82 &  0.91 &  0.94 &  0.97 &  0.98 \\
\textbf{0.75   } &   1.0 &   1.0 &   1.0 &   1.0 &   1.0 &   1.0 &   1.0 &   1.0 &   1.0 &   1.0 &   1.0 \\
\bottomrule
\end{tabular}



gamma,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gamma_0,,,,,,,,,,,
0.10,0.02,0.04,0.04,0.05,0.05,0.08,0.18,0.35,0.52,0.61,0.74
0.50,0.64,0.65,0.6,0.69,0.78,0.69,0.82,0.91,0.94,0.97,0.98
0.75,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
kendall_table = get_statistics(statistics, 'holm', 'calculate_kendall_measure', 20, 1000)
print(kendall_table.to_latex(
                #   formatters={"name": str.upper},
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                #   float_format="{:.1f}".format)
                  ))
kendall_table

\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
\textbf{gamma} &  0.0 &  0.1 &  0.2 &  0.3 &  0.4 &  0.5 &  0.6 &  0.7 &  0.8 &  0.9 &  1.0 \\
\textbf{gamma\_0} &      &      &      &      &      &      &      &      &      &      &      \\
\midrule
\textbf{0.10   } &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 \\
\textbf{0.50   } &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 \\
\textbf{0.75   } &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 \\
\bottomrule
\end{tabular}



gamma,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gamma_0,,,,,,,,,,,
0.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.50,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0.75,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
sign_table = get_statistics(statistics, 'holm', 'calculate_sign_measure', 50, 500)
print(sign_table.to_latex(
                #   formatters={"name": str.upper},
                  column_format='|c|c|c|c|c|c|c|c|c|c|c|c|',
                  bold_rows=True,
                #   float_format="{:.1f}".format)
                  ))
sign_table

\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
\textbf{gamma} &  0.0 &  0.1 &  0.2 &  0.3 &  0.4 &  0.5 &  0.6 &  0.7 &  0.8 &  0.9 &  1.0 \\
\textbf{gamma\_0} &      &      &      &      &      &      &      &      &      &      &      \\
\midrule
\textbf{0.10   } &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 \\
\textbf{0.50   } &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 &  0.0 \\
\textbf{0.75   } &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 &  1.0 \\
\bottomrule
\end{tabular}



gamma,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gamma_0,,,,,,,,,,,
0.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.75,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
for j in true_objects.keys():
    print(j)
    for i in [0.10, 0.5,0.75]:
        print(len(true_objects[j]['calculate_kendall_measure'][i][20][500][0][0])) 

holm
166
2
2
hochberg
166
2
2
benjamini_hochberg
185
190
190
